In [ ]:
# Import all required libraries
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import sympy
# from sympy import symbols, Eq, solve, simplify
import numpy as np
from utils import plot_map, graph_construction
import networkx as nx
import gurobipy as gp
from gurobipy import GRB

### QZ are circles, with (x,y) coordinates and radius (r) given.

In [ ]:
# Define constants and initial conditions
Map_qz = [(0.0, 0.0, 6.0), (12.0, 10.0, 4.0)]  # QZ circles with (x, y, radius)
# Map_qz = [(0.0, 0.0, 6.0)]  # QZ circles with (x, y, radius)

start = (-5, -5)  # Starting point
goal = (15, 15)   # Goal point

### The UAV characteristics
alpha, recharge_factor = 20, 2
beta = alpha/recharge_factor

q_min, q_max, q_start = 20, 100, 100
discritization_factor = 10
### Risk limit for each circle, temporary
Risk_limit = { circle: 30 for circle in Map_qz}

In [ ]:
### Creating the object plot_circle
### Creating a tangent object
plot_circle = plot_map(Map_qz, start, goal)
graph_object = graph_construction(Map_qz, start, goal)

### Finding the tangents
tangents_and_circles_list = graph_object.Finding_tangents()
### Finding index map and reverese index map
index_map, reverse_index_map = graph_object.create_index_map()
### Creating the visibility graph
Visibility_Graph, external_edges, internal_edges, all_edges, discrete_graph, discrete_edges_circle = graph_object.create_visibility_graph(tangents_and_circles_list, index_map, reverse_index_map)

### Changing the risk limit index
Risk_limit = { str(reverse_index_map[key]) +"_i": value for key, value in Risk_limit.items() }

In [ ]:
# for i,j, data in Visibility_Graph.edges(data=True):
#     print(i,j,data)
# plotmap = plot_map(Map_qz, start, goal)
# plotmap.plot_map_circle(tangents=True, tangent_points=[Visibility_Graph["1_o"]["2_i"]["circle2"]] )

In [ ]:
### Gurobi optimization
# all_edges = external_edges + internal_edges
model = gp.Model("Uncertain_qz")

### UAV variables, each edge
a, a_t = model.addVars(all_edges, name="a"), model.addVars(all_edges, name="a_t") ### SOC 
b = model.addVars(all_edges, lb=0, name="b")                                      ### Gas distance travelled
L = model.addVars(all_edges, name="L")                                            ### Length of the edge 
c, c_t = model.addVars(all_edges, lb= -GRB.INFINITY, name="c"), model.addVars(all_edges, lb= -GRB.INFINITY, name="c_t") ### x coor
d, d_t = model.addVars(all_edges, lb= -GRB.INFINITY, name="d"), model.addVars(all_edges, lb= -GRB.INFINITY, name="d_t") ### y coor

### UAV variables, each discretized edge inside the qz
Risk = model.addVars(discrete_edges_circle, name="R")                             ### Risk for each edge inside the qz zone

### intermediate variables
X_bar = model.addVars(all_edges, name="X_bar")
Y_bar = model.addVars(all_edges, name="Y_bar")

### Binary varibles
y = model.addVars(all_edges, vtype=gp.GRB.BINARY, name="y")             ### path selection variables
z = model.addVars(discrete_edges_circle, vtype=gp.GRB.BINARY, name="z") ### Selecting the one of the discrete edge
w = model.addVars(discrete_edges_circle, vtype=gp.GRB.BINARY, name="w")       ###  switch variable, replacement of bilinear term: z[i,j]*sw[i,j]

### Objective
model.setObjective( gp.quicksum(b[i, j] for i, j in all_edges), GRB.MINIMIZE )

### Flow constraints for
### Only for external edges
for i,j in external_edges:
    ### SOC constraint
    model.addConstr( a[i,j] - a_t[i,j] + (alpha+beta)*b[i,j] - alpha*L[i,j] == 0, f"SOC_constraint_outside_quiet_zone_{i}{j}")
    model.addConstr( b[i,j] <= L[i,j], f"Length_min_constraint_{i}{j}")

    ### length constraint
    (circle1x, circle1y, circle1r), (circle2x, circle2y, circle2r) = index_map[i], index_map[j]
    ### Intermediate variables
    X_bar[i,j] =  c[i,j] - c_t[i,j] 
    Y_bar[i,j] =  d[i,j] - d_t[i,j]
    ### Length constraint
    model.addConstr(X_bar[i,j]**2 + Y_bar[i,j]**2 <= L[i,j]**2, name=f"SOCP_constr_{i}{j}")
    model.addConstr( (c[i,j] - circle1x*y[i,j])**2 + (d[i,j] - circle1y*y[i,j])**2 <= (circle1r*y[i,j])**2, name=f"constr_c_d{i}")
    model.addConstr( (c_t[i,j] - circle2x*y[i,j])**2 + (d_t[i,j] - circle2y*y[i,j])**2 <= (circle2r*y[i,j])**2, name=f"constr_c_d_t{j}")
    
    ### Limit Constraints on the SOC variables
    model.addConstr( a[i,j] >=  q_min*y[i,j], f"a_constraints_max_{i}_{j}" )
    model.addConstr( a[i,j] <= q_max*y[i,j], f"a_constraints_min_{i}_{j}" )
    model.addConstr( a_t[i,j] >= q_min*y[i,j], f"a_t_constraints_max_{j}_{j}" )
    model.addConstr( a_t[i,j] <= q_max*y[i,j], f"a_t_constraints_min_{j}_{j}" )

    model.addConstr( c[i,j]   <= Visibility_Graph[i][j]["min_max_coordinates1"]["x_max"]*y[i,j], f"c_constraints_max_{i}_{j}" )
    model.addConstr( c[i,j]   >= Visibility_Graph[i][j]["min_max_coordinates1"]["x_min"]*y[i,j], f"c_constraints_min_{i}_{j}" )
    model.addConstr( c_t[i,j] <= Visibility_Graph[i][j]["min_max_coordinates2"]["x_max"]*y[i,j], f"c_t_constraints_max_{i}_{j}" )
    model.addConstr( c_t[i,j] >= Visibility_Graph[i][j]["min_max_coordinates2"]["x_min"]*y[i,j], f"c_t_constraints_min_{i}_{j}" )
    
    ### Limit Constraints on the SOC variables
    model.addConstr( d[i,j]   <=  Visibility_Graph[i][j]["min_max_coordinates1"]["y_max"]*y[i,j], f"d_constraints_max_{i}_{j}" )
    model.addConstr( d[i,j]   >=  Visibility_Graph[i][j]["min_max_coordinates1"]["y_min"]*y[i,j], f"d_constraints_min_{i}_{j}" )
    model.addConstr( d_t[i,j] <=  Visibility_Graph[i][j]["min_max_coordinates2"]["y_max"]*y[i,j], f"d_t_constraints_max_{j}_{i}" )
    model.addConstr( d_t[i,j] >=  Visibility_Graph[i][j]["min_max_coordinates2"]["y_min"]*y[i,j], f"d_t_constraints_min_{j}_{i}" )

# Only for internal edges, w = 1 gas mode, w = - electric mode
for i,j in internal_edges:
    radius = index_map[i][2]

    ### Length constraints
    X_bar[i,j] = c[i,j] - c_t[i,j]
    Y_bar[i,j] = d[i,j] - d_t[i,j]
    model.addConstr( X_bar[i,j]**2 + Y_bar[i,j]**2 <= gp.quicksum( z[i,k]*(discrete_graph.edges[i,k]["len_chord"])**2 for k in range(discritization_factor) ), name=f"constr_length_internal{i}")
    model.addConstr(gp.quicksum(z[i,k] for k in range(discritization_factor)) == y[i,j], name=f"constr_z{i}")

    ### Risk constraints
    # model.addConstr(gp.quicksum(w[i,k]*discrete_graph.edges[i,k]["Risk_cost"] for k in range(discritization_factor)) <= Risk_limit[i], name=f"constr_Risk{i}")
    model.addConstrs(w[i,k]*discrete_graph.edges[i,k]["Risk_cost"] <= Risk_limit[i]*z[i,k] for k in range(discritization_factor))

    (circle1x, circle1y, circle1r), (circle2x, circle2y, circle2r) = index_map[i], index_map[j]
    
    model.addConstr(L[i,j] ==  gp.quicksum(z[i,k]*discrete_graph.edges[i,k]["len_chord"] for k in range(discritization_factor)) )
    model.addConstr(y[i,j] - L[i,j]/radius >= 0 )
    model.addConstr( (c[i,j] - circle1x*y[i,j])**2 + (d[i,j] - circle1y*y[i,j])**2 <= (circle1r*y[i,j])**2, name=f"constr_c_d{i}")
    model.addConstr( (c_t[i,j] - circle2x*y[i,j])**2 + (d_t[i,j] - circle2y*y[i,j])**2 <= (circle2r*y[i,j])**2, name=f"constr_c_d_t{j}")
    
    ### SOC constraints
    model.addConstr(a[i,j] - a_t[i,j] - alpha*L[i,j]*(1 - (1+ 1/recharge_factor)*gp.quicksum(w[i,k] for k in range(discritization_factor)) ) == 0)

    model.addConstr(b[i,j] == gp.quicksum(w[i,k]*discrete_graph.edges[i,k]["len_chord"] for k in range(discritization_factor)) )

    ### limit constraints
    model.addConstrs(w[i,k] <= z[i,k] for k in range(discritization_factor))
    model.addConstrs(w[i,k] >= 0 for k in range(discritization_factor))


# ### SOC limit constraints
model.addConstr( gp.quicksum(a["s", i] for i in Visibility_Graph.successors("s")) == q_max , f"Sink_max_SOC_constraint")

model.addConstr( gp.quicksum(a_t[i, "g"] for i in Visibility_Graph.predecessors("g")) <= q_max, f"Sink_max_SOC_constraint")
model.addConstr( q_min <= gp.quicksum(a_t[i, "g"] for i in Visibility_Graph.predecessors("g")) , f"Sink_min_SOC_constraint")

# ## Constraints for all edges
model.addConstrs( gp.quicksum(y[i,j] for j in Visibility_Graph.successors(i)) == 1 for i in Visibility_Graph.nodes if i == "s")   ### Flow out from start 
model.addConstrs( gp.quicksum(y[i,j] for i in Visibility_Graph.predecessors(j)) == 1 for j in Visibility_Graph.nodes if j == "g") ### Flow in to goal
### Flow constraints for all nodes
model.addConstrs( gp.quicksum(y[i,j] for j in Visibility_Graph.successors(i)) == gp.quicksum(y[j,i] for j in Visibility_Graph.predecessors(i)) for i in Visibility_Graph.nodes if i != "s" and i != "g")
model.addConstrs( gp.quicksum(y[i,j] for j in Visibility_Graph.successors(i)) <= 1 for i in Visibility_Graph.nodes if i != "s" and i != "g")
model.addConstrs( gp.quicksum(y[j,i] for j in Visibility_Graph.predecessors(i)) <= 1 for i in Visibility_Graph.nodes if i != "s" and i != "g")

### Flow constraints for  all nodes along with the source node for a
model.addConstrs( gp.quicksum(a[i,j] for j in Visibility_Graph.successors(i)) - gp.quicksum(a_t[j,i] for j in Visibility_Graph.predecessors(i)) == 0 for i in Visibility_Graph.nodes if i != "s" and i != "g")
### Flow constraints for all nodes along with the source node for c
model.addConstrs( gp.quicksum(c[i,j] for j in Visibility_Graph.successors(i)) - gp.quicksum(c_t[j,i] for j in Visibility_Graph.predecessors(i)) == 0 for i in Visibility_Graph.nodes if i != "s" and i != "g" )


model_params={"Non_Convex": -1, "MIPgap": 0.005, "TimeLimit": 1.5, "ScaleFlag":0, "OutputFlag": 1}
### Set it to -1 or 2 so that it can use non-convex formulation
model.setParam('OutputFlag', 0)
# model.setParam("NonConvex", model_params["Non_Convex"])
model.setParam('MIPGap', model_params["MIPgap"])
model.setParam("Timelimit", model_params["TimeLimit"])
model.setParam("OutputFlag", model_params["OutputFlag"])
# model.setParam("PSDTol", model_params["PSDTol"])
# model.setParam("ScaleFlag", 0)
model.write('test.lp')

model.optimize()

### Code that is not needed for now

In [ ]:
def SequencePath(path_index):
    # Find the starting tuple
    start_tuple = next(t for t in path_index if t[0] == "s")

    # Initialize the result list
    result = [start_tuple]
    path_index.remove(start_tuple)

    # Find the next tuple
    while True:
        next_tuple = next((t for t in path_index if t[0] == result[-1][1]), None)
        if next_tuple is None:
            break
        result.append(next_tuple)
        path_index.remove(next_tuple)
    return result

def extract_variable_values(my_list, path_index, var):

    if var == 'c' or var == 'd':    
        var_list = {}
        for variable in my_list:
            if variable[0][0] == var and variable[0][1] == "[":
                a = variable[0][2:-1].split(",")
                a = [a[0],a[1]]
                print(a)
                if a in path_index:
                    var_list[a[0]] = variable[1]
            
            elif variable[0][0:3] == f'{var}_t':
                a = variable[0][4:-1].split(",")
                a = [a[0], a[1]]
                if a in path_index:
                    var_list[a[1]] = variable[1]
        return var_list    

In [ ]:
my_dict = {"Vars": []}
if model.status == gp.GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    optimal_cost = model.ObjVal
    print(optimal_cost)    
    for v in model.getVars():
        # print(v.VarName,v.x)
        my_dict["Vars"].append((v.VarName,v.x))

x_coor = []
y_coor = []
indexes = []
for key, value in my_dict.items():
    for item in value:
        if item[0][0] == "c" and item[1] != 0:
            x_coor.append(item)
            a = item[0][2:-1].split(",")
            indexes.append(a)
        if item[0][0] == "d" and item[1] != 0:
            y_coor.append(item)

ordered_path = SequencePath(indexes)
x_coor = extract_variable_values(x_coor, ordered_path, var= "c")
y_coor = extract_variable_values(y_coor, ordered_path, var= "d")

In [ ]:
x_coor = [x_coor[key[0]] for key in ordered_path if key[0] in x_coor]
y_coor = [y_coor[key[0]] for key in ordered_path if key[0] in y_coor]

x_coor.append(goal[0])
y_coor.append(goal[1])

In [ ]:
plotmap = plot_map(Map_qz, start, goal)
plotmap.plot_path([x_coor, y_coor] )